***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@stat.sinica.edu.tw </p>
### <p> Institute of Statistical Science, Academia Sinica, Taipei, Taiwan.</p>  
---

# Input

In [1]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime

In [2]:
# Input
TRIALS    = 1
timestr   = ''
#Region_Index_loc = 6 # column location; Others is 6
REGION_INDEX_LOC = 4 # column location; MNIST is 4

PATH1='../data/MNIST_seedinds.txt'
PATH2='../data/MNIST_bilabels.txt'
PATH3='../data/MNIST_seedinds_neighborregions.txt'
PATH4='../data/MNIST_Labels_Spec20.csv'
PATH5='../data/Small_MNIST_tSNE_embeddings.pickle'
#PATH5='../phase3_data/MNIST_Labels_5000.csv'
#=================================================================

# (1)PATH1
df = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df = pandas.read_csv(PATH4)
#1213 add auto judge
all_region_index = df.to_numpy().T[REGION_INDEX_LOC].astype(int)

region: [15  9 11  6  7  8 10 20 14 12 19  1 13 18  4]
region amount: 15


# Get neighbors

In [3]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  15
[[ 9]
 [15]
 [16]
 [ 7]
 [ 6]
 [ 8]
 [20]
 [10]
 [ 5]
 [ 6]
 [ 1]
 [19]
 [13]
 [ 4]
 [18]]
neighbor amount:  14


[[9],
 [15],
 [16],
 [7],
 [6],
 [8],
 [20],
 [10],
 [5],
 [],
 [1],
 [19],
 [13],
 [4],
 [18]]

In [4]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[15, 9],
 [9, 15],
 [11, 16],
 [6, 7],
 [7, 6],
 [8, 8],
 [10, 20],
 [20, 10],
 [14, 5],
 [12],
 [19, 1],
 [1, 19],
 [13, 13],
 [18, 4],
 [4, 18]]

In [5]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

** original duplicate at i= 5 ;region 8 ;duplicate size 491
** same duplicate situation
** original duplicate at i= 12 ;region 13 ;duplicate size 460
** same duplicate situation


In [6]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (5000, 3)
training size: 7151


# CNN original

In [7]:
#======================================================

#1213 add auto judge
from CNN_Modules_1D import ME_CNN

# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

n, p1, p2 0 0 0


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)               

6435/6435 [==============================] - 0s 46us/step - loss: 1.4661 - accuracy: 0.3284 - val_loss: 1.4780 - val_accuracy: 0.3268
Epoch 48/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4609 - accuracy: 0.3285 - val_loss: 1.4916 - val_accuracy: 0.3310
Epoch 49/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4589 - accuracy: 0.3315 - val_loss: 1.4845 - val_accuracy: 0.3198
Epoch 50/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4559 - accuracy: 0.3372 - val_loss: 1.4811 - val_accuracy: 0.3366
Epoch 51/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4576 - accuracy: 0.3279 - val_loss: 1.4990 - val_accuracy: 0.2989
Epoch 52/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4541 - accuracy: 0.3336 - val_loss: 1.4681 - val_accuracy: 0.3240
Epoch 53/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4551 - accuracy: 0.3402 - val_loss: 1.4722 - val_accurac

6435/6435 [==============================] - 0s 37us/step - loss: 1.6503 - accuracy: 0.2990 - val_loss: 1.6171 - val_accuracy: 0.3073
Epoch 9/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.6292 - accuracy: 0.3049 - val_loss: 1.5737 - val_accuracy: 0.3408
Epoch 10/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.5997 - accuracy: 0.3142 - val_loss: 1.5601 - val_accuracy: 0.3226
Epoch 11/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.5708 - accuracy: 0.3173 - val_loss: 1.5274 - val_accuracy: 0.3310
Epoch 12/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.5504 - accuracy: 0.3183 - val_loss: 1.5171 - val_accuracy: 0.3422
Epoch 13/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.5412 - accuracy: 0.3100 - val_loss: 1.5046 - val_accuracy: 0.3198
Epoch 14/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.5287 - accuracy: 0.3119 - val_loss: 1.4964 - val_accuracy

Epoch 64/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.3662 - accuracy: 0.3879 - val_loss: 1.3494 - val_accuracy: 0.3883
Epoch 65/80
6435/6435 [==============================] - 0s 45us/step - loss: 1.3585 - accuracy: 0.3767 - val_loss: 1.3413 - val_accuracy: 0.3966
Epoch 66/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.3509 - accuracy: 0.3809 - val_loss: 1.3495 - val_accuracy: 0.3603
Epoch 67/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.3362 - accuracy: 0.3806 - val_loss: 1.3226 - val_accuracy: 0.3813
Epoch 68/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.3296 - accuracy: 0.3784 - val_loss: 1.3278 - val_accuracy: 0.3911
Epoch 69/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.3224 - accuracy: 0.3866 - val_loss: 1.3290 - val_accuracy: 0.3617
Epoch 70/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.3198 - accuracy: 0.3832 - val_loss: 1.3098 -

6435/6435 [==============================] - 0s 39us/step - loss: 1.5088 - accuracy: 0.3057 - val_loss: 1.4524 - val_accuracy: 0.3226
Epoch 26/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.5036 - accuracy: 0.3148 - val_loss: 1.4335 - val_accuracy: 0.3520
Epoch 27/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.5036 - accuracy: 0.3068 - val_loss: 1.4376 - val_accuracy: 0.3142
Epoch 28/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4992 - accuracy: 0.3184 - val_loss: 1.4243 - val_accuracy: 0.3338
Epoch 29/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4997 - accuracy: 0.3209 - val_loss: 1.4576 - val_accuracy: 0.3212
Epoch 30/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.5051 - accuracy: 0.3122 - val_loss: 1.4191 - val_accuracy: 0.3478
Epoch 31/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.5078 - accuracy: 0.3040 - val_loss: 1.4351 - val_accurac

6435/6435 [==============================] - 0s 39us/step - loss: 1.4741 - accuracy: 0.3232 - val_loss: 1.4923 - val_accuracy: 0.3352
Epoch 34/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4681 - accuracy: 0.3253 - val_loss: 1.5127 - val_accuracy: 0.2933
Epoch 35/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4702 - accuracy: 0.3271 - val_loss: 1.4882 - val_accuracy: 0.3324
Epoch 36/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4693 - accuracy: 0.3305 - val_loss: 1.4715 - val_accuracy: 0.3296
Epoch 37/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4592 - accuracy: 0.3270 - val_loss: 1.4828 - val_accuracy: 0.3282
Epoch 38/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4554 - accuracy: 0.3336 - val_loss: 1.4795 - val_accuracy: 0.3142
Epoch 39/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4532 - accuracy: 0.3444 - val_loss: 1.4733 - val_accurac

Train on 6435 samples, validate on 716 samples
Epoch 1/80
6435/6435 [==============================] - 0s 76us/step - loss: 2.6763 - accuracy: 0.1189 - val_loss: 2.5966 - val_accuracy: 0.1229
Epoch 2/80
6435/6435 [==============================] - 0s 47us/step - loss: 2.3913 - accuracy: 0.1360 - val_loss: 2.1923 - val_accuracy: 0.1564
Epoch 3/80
6435/6435 [==============================] - 0s 42us/step - loss: 2.0523 - accuracy: 0.2239 - val_loss: 1.9628 - val_accuracy: 0.2626
Epoch 4/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.8973 - accuracy: 0.2810 - val_loss: 1.8647 - val_accuracy: 0.2654
Epoch 5/80
6435/6435 [==============================] - 0s 43us/step - loss: 1.8025 - accuracy: 0.2985 - val_loss: 1.8074 - val_accuracy: 0.2821
Epoch 6/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.7286 - accuracy: 0.2960 - val_loss: 1.7062 - val_accuracy: 0.2933
Epoch 7/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.6598

In [8]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

(5, 5000) (5, 5000, 15)


# CNN combination and removal

In [9]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

105

n, p1, p2 0 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84

6435/6435 [==============================] - 0s 39us/step - loss: 1.4703 - accuracy: 0.3175 - val_loss: 1.4391 - val_accuracy: 0.3212
Epoch 48/80
6435/6435 [==============================] - 0s 43us/step - loss: 1.4688 - accuracy: 0.3232 - val_loss: 1.4395 - val_accuracy: 0.3198
Epoch 49/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4679 - accuracy: 0.3198 - val_loss: 1.4372 - val_accuracy: 0.3073
Epoch 50/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4726 - accuracy: 0.3251 - val_loss: 1.4395 - val_accuracy: 0.3212
Epoch 51/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4704 - accuracy: 0.3153 - val_loss: 1.4383 - val_accuracy: 0.3059
Epoch 52/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4663 - accuracy: 0.3178 - val_loss: 1.4355 - val_accuracy: 0.3198
Epoch 53/80
6435/6435 [==============================] - 0s 48us/step - loss: 1.4632 - accuracy: 0.3249 - val_loss: 1.4360 - val_accurac

6435/6435 [==============================] - 0s 39us/step - loss: 1.4879 - accuracy: 0.3574 - val_loss: 1.4670 - val_accuracy: 0.3659
Epoch 9/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4714 - accuracy: 0.3518 - val_loss: 1.4486 - val_accuracy: 0.3687
Epoch 10/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4536 - accuracy: 0.3543 - val_loss: 1.4397 - val_accuracy: 0.3659
Epoch 11/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4385 - accuracy: 0.3526 - val_loss: 1.4276 - val_accuracy: 0.3520
Epoch 12/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4293 - accuracy: 0.3594 - val_loss: 1.4203 - val_accuracy: 0.3506
Epoch 13/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4254 - accuracy: 0.3599 - val_loss: 1.4080 - val_accuracy: 0.3883
Epoch 14/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4144 - accuracy: 0.3616 - val_loss: 1.4098 - val_accuracy

6435/6435 [==============================] - 0s 37us/step - loss: 1.1373 - accuracy: 0.4370 - val_loss: 1.1240 - val_accuracy: 0.4525
Epoch 65/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.1389 - accuracy: 0.4502 - val_loss: 1.1196 - val_accuracy: 0.4679
Epoch 66/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.1398 - accuracy: 0.4427 - val_loss: 1.1084 - val_accuracy: 0.4707
Epoch 67/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.1342 - accuracy: 0.4378 - val_loss: 1.1215 - val_accuracy: 0.4553
Epoch 68/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.1333 - accuracy: 0.4395 - val_loss: 1.1150 - val_accuracy: 0.4441
Epoch 69/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.1355 - accuracy: 0.4409 - val_loss: 1.1127 - val_accuracy: 0.4735
Epoch 70/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.1296 - accuracy: 0.4432 - val_loss: 1.1117 - val_accurac

...

6435/6435 [==============================] - 0s 40us/step - loss: 2.0406 - accuracy: 0.2384 - val_loss: 1.9770 - val_accuracy: 0.2514
Epoch 4/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.9067 - accuracy: 0.2446 - val_loss: 1.9074 - val_accuracy: 0.2081
Epoch 5/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.8364 - accuracy: 0.2458 - val_loss: 1.8472 - val_accuracy: 0.2360
Epoch 6/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.7807 - accuracy: 0.2679 - val_loss: 1.8111 - val_accuracy: 0.2486
Epoch 7/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.7496 - accuracy: 0.2808 - val_loss: 1.7614 - val_accuracy: 0.2668
Epoch 8/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.7220 - accuracy: 0.2827 - val_loss: 1.7563 - val_accuracy: 0.2835
Epoch 9/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.7026 - accuracy: 0.2942 - val_loss: 1.7387 - val_accuracy: 0.2

6435/6435 [==============================] - 0s 40us/step - loss: 1.7342 - accuracy: 0.3178 - val_loss: 1.7226 - val_accuracy: 0.3226
Epoch 7/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.6778 - accuracy: 0.3170 - val_loss: 1.6535 - val_accuracy: 0.3226
Epoch 8/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.6247 - accuracy: 0.3263 - val_loss: 1.6174 - val_accuracy: 0.3212
Epoch 9/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.5872 - accuracy: 0.3139 - val_loss: 1.5838 - val_accuracy: 0.3212
Epoch 10/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.5609 - accuracy: 0.3212 - val_loss: 1.5586 - val_accuracy: 0.3268
Epoch 11/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.5480 - accuracy: 0.3274 - val_loss: 1.5491 - val_accuracy: 0.3296
Epoch 12/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.5327 - accuracy: 0.3279 - val_loss: 1.5352 - val_accuracy: 

6435/6435 [==============================] - 0s 37us/step - loss: 1.3456 - accuracy: 0.3935 - val_loss: 1.3591 - val_accuracy: 0.3827
Epoch 63/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.3342 - accuracy: 0.3826 - val_loss: 1.3506 - val_accuracy: 0.3757
Epoch 64/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.3176 - accuracy: 0.3960 - val_loss: 1.3457 - val_accuracy: 0.3785
Epoch 65/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.3162 - accuracy: 0.3891 - val_loss: 1.3302 - val_accuracy: 0.3771
Epoch 66/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.3039 - accuracy: 0.3944 - val_loss: 1.3493 - val_accuracy: 0.3953
Epoch 67/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.3026 - accuracy: 0.3882 - val_loss: 1.3139 - val_accuracy: 0.3687
Epoch 68/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.2904 - accuracy: 0.3943 - val_loss: 1.3135 - val_accurac

6435/6435 [==============================] - 0s 39us/step - loss: 1.4552 - accuracy: 0.3277 - val_loss: 1.4499 - val_accuracy: 0.3324
Epoch 24/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4550 - accuracy: 0.3260 - val_loss: 1.4417 - val_accuracy: 0.3128
Epoch 25/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4581 - accuracy: 0.3211 - val_loss: 1.4611 - val_accuracy: 0.3226
Epoch 26/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4505 - accuracy: 0.3262 - val_loss: 1.4350 - val_accuracy: 0.3282
Epoch 27/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4442 - accuracy: 0.3321 - val_loss: 1.4281 - val_accuracy: 0.3296
Epoch 28/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4403 - accuracy: 0.3340 - val_loss: 1.4530 - val_accuracy: 0.3338
Epoch 29/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4425 - accuracy: 0.3319 - val_loss: 1.4511 - val_accurac

KeyboardInterrupt: 

In [ ]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

In [ ]:
print("done")